<a href="https://colab.research.google.com/github/KiarashS/WeakMedSAM/blob/main/WeakMedSAM_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
print(torch.__version__)

2.6.0+cu124


In [3]:
import torchvision
print(torchvision.__version__)

0.21.0+cu124


In [ ]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !conda install pytorch torchvision cudatoolkit=11.8 -c pytorch

In [ ]:
!python -m pip install SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 14.3 MB/s eta 0:00:00


In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kiarashs","key":"dd43ac962ffe7d1775c5b38aface305d"}'}

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets list

In [ ]:
!kaggle datasets download 'aryashah2k/brain-tumor-segmentation-brats-2019'

Dataset URL: https://www.kaggle.com/datasets/aryashah2k/brain-tumor-segmentation-brats-2019
License(s): CC0-1.0
 98% 2.56G/2.60G [00:22<00:01, 29.4MB/s]
100% 2.60G/2.60G [00:22<00:00, 126MB/s] 


In [ ]:
!git clone https://github.com/KiarashS/WeakMedSAM.git

Cloning into 'WeakMedSAM'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 59 (delta 13), reused 41 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 79.92 KiB | 4.21 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [ ]:
!mkdir WeakMedSAM/brats-input

In [ ]:
%%capture
!unzip brain-tumor-segmentation-brats-2019.zip -d WeakMedSAM/brats-input

In [ ]:
!mv WeakMedSAM/brats-input/MICCAI_BraTS_2019_Data_Training/* WeakMedSAM/brats-input

In [ ]:
!rm -rf WeakMedSAM/brats-output/

In [ ]:
!python WeakMedSAM/brats/preprocess.py --input-path WeakMedSAM/brats-input/ --output-path WeakMedSAM/brats-output/

  7%|████▍                                                         | 24/335 [00:05<01:07,  4.61it/s]

In [ ]:
!pip install scikit-learn

In [ ]:
%%capture
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
#!pip install --upgrade torch torchvision

In [ ]:
# %pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117

In [ ]:
!python WeakMedSAM/cluster.py --batch_size 256 --data_path WeakMedSAM/brats-output --data_module brats --parent_classes 1 --child_classes 8 --save_path WeakMedSAM/cluster-file --gpus 1

In [ ]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

In [ ]:
# import tensorflow as tf
# tf.test.gpu_device_name()

In [ ]:
!python WeakMedSAM/train.py --seed 42 --sam_ckpt sam_vit_b_01ec64.pth --lr '1e-4' --batch_size 24 --max_epochs 10 --val_iters 3000 --index mymodelindex --data_path WeakMedSAM/brats-output/ --data_module brats --parent_classes 1 --child_classes 8 --child_weight 0.5 --cluster_file WeakMedSAM/cluster-file/brats-8.bin --logdir WeakMedSAM/logdir --gpus 1

In [ ]:
!python WeakMedSAM/lab_gen.py --batch-size 24 --data-path WeakMedSAM/brats_output --save-path WeakMedSAM/pseudo_label --data-module brats --parent-classes 1 --child-classes 8 --samus-ckpt WeakMedSAM/logdir/mymodelindex/mymodelindex.pth --sam-ckpt sam_vit_b_01ec64.pth --t 4 --beta 4 --threshold 0.5 --gpus 2

In [ ]:
!python WeakMedSAM/train_unet.py --seed 42 --lr '1e-4' --batch_size 128 --max_epochs 10 --val_iters 500 --index mymodelindex --data_path WeakMedSAM/brats_output --lab_path WeakMedSAM/pseudo_label --data_module brats --num_classes 2 --logdir WeakMedSAM/logdir --gpus 2

In [ ]:
!python WeakMedSAM/eval.py --data_path WeakMedSAM/brats_output --data_module 'brats' --batch_size 128 --num_classes 2 --ckpt WeakMedSAM/logdir/mymodelindex/mymodelindex.pth --gpus 2